# Using Recursive Feature Selection (RFS) to find: 
1. the most important feature to predict: (A) is_recid, (B) two_year_recid using logistic Regression
2. the combination of the most important features (2~3) to redict (A) is_recid, (B) two_year_recid


### TO DO: RFS with Neural Network 


"We compared the recidivism risk categories predicted by the COMPAS tool to the actual recidivism rates of defendants in the two years after they were scored, and found that the score correctly predicted an offender’s recidivism 61 percent of the time, but was only correct in its predictions of violent recidivism 20 percent of the time."

In [13]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [14]:
compas = pd.read_csv('data/compas-scores-two-years-violent.csv')
compas.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid,two_year_recid.1
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1,1
2,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0,0
3,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,Low,2013-03-26,NaN,NaN,2,0,1102,0,0,0
4,7,marsha miles,marsha,miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,...,Low,2013-11-30,2013-11-30,2013-12-01,0,1,853,0,0,0


In [15]:
compas['c_jail_in'] = pd.to_datetime(compas['c_jail_in'], errors='coerce')
compas['c_jail_out'] = pd.to_datetime(compas['c_jail_out'], errors='coerce')

compas['v_screening_date'] = pd.to_datetime(compas['v_screening_date'], errors='coerce')
compas['vr_offense_date'] = pd.to_datetime(compas['vr_offense_date'], errors='coerce')

compas['c_offense_date'] = pd.to_datetime(compas['c_offense_date'], errors='coerce')
compas['c_arrest_date'] = pd.to_datetime(compas['c_arrest_date'], errors='coerce')

#compas['compas_screening_date'] = pd.to_datetime(compas['compas_screening_date'], format='mixed')

In [16]:
compas = compas.drop(['first','last','c_case_number','c_charge_degree', 'c_charge_desc', 'r_case_number','r_charge_degree','vr_case_number','vr_charge_degree','start','end','event'],axis=1)

#remove row with nan scores 
compas = compas.loc[compas['score_text'].notnull()]
#clean out the "-1" decile score from dataset 
compas = compas.loc[compas['v_decile_score'] != -1]

#label encodimg
compas['sex'] = compas['sex'].replace({'Male': 1, 'Female': 0})
compas['score_text'] = compas['score_text'].replace({'Low':0, 'Medium':1, 'High': 2})

#days in jail
compas['jail_out- jail_in'] = (compas['c_jail_out'] - compas['c_jail_in']).dt.days

In [17]:
## features in consideration: 
'''
Data columns (total 43 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----                  
 3   sex                      4738 non-null   int64             
 5   age                      4738 non-null   int64         
 6   age_cat                  4738 non-null   object        
 7   race                     4738 non-null   object        
 8   juv_fel_count            4738 non-null   int64         
 9   decile_score             4738 non-null   int64         
 10  juv_misd_count           4738 non-null   int64         
 11  juv_other_count          4738 non-null   int64         
 12  priors_count             4738 non-null   int64             
 18  c_days_from_compas       4727 non-null   float64       
 19  is_recid                 4738 non-null   int64                    
 25  violent_recid            0 non-null      float64       
 26  is_violent_recid         4738 non-null   int64           
 30  decile_score.1           4738 non-null   int64         
 31  score_text               4738 non-null   int64              
 34  v_decile_score           4738 non-null   int64         
 35  v_score_text             4738 non-null   object        
 39  priors_count.1           4738 non-null   int64         
 40  two_year_recid           4738 non-null   int64         
 41  two_year_recid.1         4738 non-null   int64         
 42  jail_out- jail_in        4470 non-null   float64     
'''

'\nData columns (total 43 columns):\n #   Column                   Non-Null Count  Dtype         \n---  ------                   --------------  -----                  \n 3   sex                      4738 non-null   int64             \n 5   age                      4738 non-null   int64         \n 6   age_cat                  4738 non-null   object        \n 7   race                     4738 non-null   object        \n 8   juv_fel_count            4738 non-null   int64         \n 9   decile_score             4738 non-null   int64         \n 10  juv_misd_count           4738 non-null   int64         \n 11  juv_other_count          4738 non-null   int64         \n 12  priors_count             4738 non-null   int64             \n 18  c_days_from_compas       4727 non-null   float64       \n 19  is_recid                 4738 non-null   int64                    \n 25  violent_recid            0 non-null      float64       \n 26  is_violent_recid         4738 non-null   int64           \n 30

## RFS with target as ProPublica Two Year Recidivism 

In [18]:

feature_list = ['sex','age','age_cat','race','juv_fel_count','juv_misd_count',
                      'juv_other_count','priors_count','c_days_from_compas' ,
                      'priors_count.1', 'jail_out- jail_in']
all_feature = compas[feature_list]

#clean and label features 
#all_feature.loc[:,'v_score_text'] = all_feature['v_score_text'].replace({'Low':0, 'Medium':1, 'High': 2})
all_feature.loc[:, 'race'] = LabelEncoder().fit_transform(all_feature['race'])
all_feature.loc[:,'age_cat'] = all_feature['age_cat'].replace({'Greater than 45': 2, '25 - 45': 1, 'Less than 25': 0})

In [19]:
#feature = all_features
target = compas['two_year_recid']

imputer = SimpleImputer(strategy='mean')
features_imputed = imputer.fit_transform(all_feature)

train_features, test_features, train_target, test_target = train_test_split(features_imputed, target, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=1000)

# RFE to select features
rfe = RFE(model, n_features_to_select=5) 
fit = rfe.fit(train_features, train_target)

selected_features = pd.DataFrame({'Feature': feature_list, 'Ranking': fit.ranking_})
selected_features = selected_features.sort_values(by='Ranking')
print(selected_features)
selected_features = selected_features[selected_features['Ranking'] == 1]['Feature']
selected_features = list(selected_features)

# Train the logistic regression model with selected features
model.fit(train_features[:, fit.support_], train_target)

# get coeff
feature_weights = pd.DataFrame({'Feature': selected_features, 'Weight': model.coef_[0]})
feature_weights = feature_weights.sort_values(by='Weight', ascending=False)

# feature weights
print("Feature Weights:")
print(feature_weights)
print()
predicted_classes = model.predict(test_features[:, fit.support_])


# Evaluating the model
accuracy = accuracy_score(test_target, predicted_classes)
report = classification_report(test_target, predicted_classes)
conf_matrix = confusion_matrix(test_target, predicted_classes)

print(f"Selected Features: {selected_features}")
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

               Feature  Ranking
0                  sex        1
2              age_cat        1
5       juv_misd_count        1
6      juv_other_count        1
9       priors_count.1        1
3                 race        2
4        juv_fel_count        3
7         priors_count        4
1                  age        5
10   jail_out- jail_in        6
8   c_days_from_compas        7
Feature Weights:
           Feature    Weight
0              sex  0.576902
2   juv_misd_count  0.147465
4   priors_count.1  0.133989
3  juv_other_count  0.110289
1          age_cat -0.837071

Selected Features: ['sex', 'age_cat', 'juv_misd_count', 'juv_other_count', 'priors_count.1']
Accuracy: 0.8459915611814346
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.99      0.91       791
           1       0.70      0.12      0.21       157

    accuracy                           0.85       948
   macro avg       0.78      0.56      0.56       948
weight

## RFS with target as COMPAS Violent Decile Score 

In [20]:

feature_list = ['sex','age','age_cat','race','juv_fel_count','juv_misd_count',
                      'juv_other_count','priors_count','c_days_from_compas',
                      'priors_count.1', 'jail_out- jail_in']

all_feature = compas[feature_list]
#clean and label features 
all_feature.loc[:, 'race'] = LabelEncoder().fit_transform(all_feature['race'])
all_feature.loc[:,'age_cat'] = all_feature['age_cat'].replace({'Greater than 45': 2, '25 - 45': 1, 'Less than 25': 0})

In [21]:
#feature = all_features

compas['reoffend'] = compas['v_decile_score'].apply(lambda x: 1 if x > 5 else 0)
target = compas['reoffend']


imputer = SimpleImputer(strategy='mean') 
features_imputed = imputer.fit_transform(all_feature)

train_features, test_features, train_target, test_target = train_test_split(features_imputed, target, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=1000)

# Using RFE to select features
rfe = RFE(model, n_features_to_select=5) # change this
fit = rfe.fit(train_features, train_target)

# Printing features rank 
selected_features = pd.DataFrame({'Feature': feature_list, 'Ranking': fit.ranking_})
selected_features = selected_features.sort_values(by='Ranking')
print(selected_features)
selected_features = selected_features[selected_features['Ranking'] == 1]['Feature']
selected_features = list(selected_features)

# logistic regression model 
model.fit(train_features[:, fit.support_], train_target)

# get coeff
feature_weights = pd.DataFrame({'Feature': selected_features, 'Weight': model.coef_[0]})
feature_weights = feature_weights.sort_values(by='Weight', ascending=False)

# feature weights
print()
print("Feature Weights:")
print(feature_weights)
print()

predicted_classes = model.predict(test_features[:, fit.support_])

accuracy = accuracy_score(test_target, predicted_classes)
report = classification_report(test_target, predicted_classes, zero_division=1)
conf_matrix = confusion_matrix(test_target, predicted_classes)

print(f"Selected Features: {selected_features}")
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

               Feature  Ranking
0                  sex        1
2              age_cat        1
3                 race        1
4        juv_fel_count        1
9       priors_count.1        1
1                  age        2
7         priors_count        3
5       juv_misd_count        4
6      juv_other_count        5
10   jail_out- jail_in        6
8   c_days_from_compas        7

Feature Weights:
          Feature    Weight
0             sex  0.757818
3   juv_fel_count  0.431145
4  priors_count.1  0.150832
2            race -0.256325
1         age_cat -2.348358

Selected Features: ['sex', 'age_cat', 'race', 'juv_fel_count', 'priors_count.1']
Accuracy: 0.8270042194092827
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.91      0.89       765
           1       0.56      0.48      0.51       183

    accuracy                           0.83       948
   macro avg       0.72      0.69      0.70       948
weighted avg       0.82

In [34]:

compas['reoffend'] = compas['v_decile_score'].apply(lambda x: 1 if x > 5 else 0)
compas.loc[:, 'race'] = LabelEncoder().fit_transform(compas['race'])
compas.loc[:,'age_cat'] = compas['age_cat'].replace({'Greater than 45': 2, '25 - 45': 1, 'Less than 25': 0})

feature_list = ['sex', 'age', 'race', 'juv_fel_count', 'juv_misd_count',
                'juv_other_count', 'priors_count', 'c_days_from_compas',
                'priors_count.1', 'jail_out- jail_in']

features = compas[feature_list]
target = compas['reoffend']

imputer = SimpleImputer(strategy='mean') 
features_imputed = imputer.fit_transform(features)

train_features, test_features, train_target, test_target = train_test_split(
    features_imputed, target, test_size=0.2, random_state=42
)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
sfm = SelectFromModel(rf_classifier, threshold='median', max_features=3)
sfm.fit(train_features, train_target)

selected_features = [feature_list[i] for i, selected in enumerate(sfm.get_support()) if selected]
print("Selected Features:", selected_features)


Selected Features: ['age', 'c_days_from_compas', 'jail_out- jail_in']


In [37]:
model_selected = Sequential()
model_selected.add(Dense(10, input_dim=len(selected_features), activation='relu'))  # Input layer and first hidden layer
model_selected.add(Dense(1, activation='sigmoid'))  # Output layer

# Compile the model
model_selected.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model on the selected features
model_selected.fit(train_features[selected_features], train_target, epochs=100, batch_size=10, validation_split=0.1)

# Evaluate the model on the selected features
accuracy_selected = model_selected.evaluate(test_features[selected_features], test_target)[1]
print(f'Accuracy on the selected feature set: {accuracy_selected}')

# Make predictions
new_data = pd.DataFrame({
    'age': [25, 40],
    'c_days_from_compas': [40, 60],
    'jail_out- jail_in': [50, 0]
})

predictions = model_selected.predict(new_data[selected_features])
print(predictions)

Epoch 1/100
342/342 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.7933 - val_loss: nan - val_accuracy: 0.7731
Epoch 2/100
342/342 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.7948 - val_loss: nan - val_accuracy: 0.7731
Epoch 3/100
342/342 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.7948 - val_loss: nan - val_accuracy: 0.7731
Epoch 4/100
342/342 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.7948 - val_loss: nan - val_accuracy: 0.7731
Epoch 5/100
342/342 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.7948 - val_loss: nan - val_accuracy: 0.7731
Epoch 6/100
342/342 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.7948 - val_loss: nan - val_accuracy: 0.7731
Epoch 7/100
342/342 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.7948 - val_loss: nan - val_accuracy: 0.7731
Epoch 8/100
342/342 [====================